# Projet 6 : Analysez les ventes d'une librairie

In [151]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

##  Import des datasets 

On dispose de 3 fichiers de données : clients, produits et transactions

In [152]:
# import des fichiers CSV extraits de la base de données de la librairie
df_transactions = pd.read_csv('C:/Users/p092279/Desktop/Documents/Data Analyst/P6/DAN-P6-donnees/transactions.csv') #les ventes (appelées “Transactions”)
df_products = pd.read_csv('C:/Users/p092279/Desktop/Documents/Data Analyst/P6/DAN-P6-donnees/products.csv') #la liste des produits
df_customers = pd.read_csv('C:/Users/p092279/Desktop/Documents/Data Analyst/P6/DAN-P6-donnees/customers.csv') #la liste des clients

In [153]:
#Visualisation rapide de la dataframe transaction
df_transactions.head()

,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232


In [154]:
# Statistique dataframe transaction
df_transactions.describe()

,id_prod,date,session_id,client_id
count,679532,679532,679532,679532
unique,3267,679371,342316,8602
top,1_369,test_2021-03-01 02:30:02.237413,s_0,c_1609
freq,2252,13,200,25488


Le DataFrame des transactions contient quatre colonnes : l'identifiant des produits, la date de la transaction, le numéro de la session et l'identifiant du client. Nous avons 3267 produits distincts répertoriés dans le dataframe.

In [155]:
#Visualisation rapide de la dataframe produits
df_products.head()

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0


In [156]:
#Statistique du dataset produits
df_products.describe()

,price,categ
count,3287.000000,3287.000000
mean,21.856641,0.370246
std,29.847908,0.615387
min,-1.000000,0.000000
25%,6.990000,0.000000
50%,13.060000,0.000000
75%,22.990000,1.000000
max,300.000000,2.000000


Le dataset des produits contient deux colonnes : le prix de chaque produit dont la valeur min est -1 et la valeur max 300, ainsi que la categorie des produits. Dans la suite de notre analyse, nous reviendrons sur la valeur négative de la variable "price".

In [157]:
#Visualisation rapide de la dataframe clients
df_customers.head()

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943


In [158]:
#Stattistique du dataset clients
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.2+ KB


Le dataset des clients contient trois colonnes : l'identifiant de chaque client, le genre (masculin ou feminin) ainsi que l'année de naissance.

# Mission_1 : Nettoyage des données

## Nettoyage du fichier des transactions

In [159]:
# Les 5 premieres lignes du fichier des transactions
df_transactions.head()

,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232


In [160]:
# les données manquantes du fichier
df_transactions.isna().sum()

id_prod       0
date          0
session_id    0
client_id     0
dtype: int64

Le fichier ne présente pas de données manquantes.

In [161]:
# création d'une variable contenant les informations dupliquées
df_duplicated_transactions = df_transactions.duplicated()
df_transactions[df_duplicated_transactions]

,id_prod,date,session_id,client_id
27778,T_0,test_2021-03-01 02:30:02.237437,s_0,ct_1
52424,T_0,test_2021-03-01 02:30:02.237419,s_0,ct_0
96687,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_1
130188,T_0,test_2021-03-01 02:30:02.237419,s_0,ct_0
139339,T_0,test_2021-03-01 02:30:02.237443,s_0,ct_1
...,...,...,...,...
653098,T_0,test_2021-03-01 02:30:02.237432,s_0,ct_0
657830,T_0,test_2021-03-01 02:30:02.237417,s_0,ct_0
662081,T_0,test_2021-03-01 02:30:02.237427,s_0,ct_1
671647,T_0,test_2021-03-01 02:30:02.237424,s_0,ct_1


on remarque 126 lignes correspondant à des sessions de tests qu'il faudra supprimer du dataset. Il faudra enlever aussi le produit avec l'identifiant "id_prod = T_0" dans le fichier des produits et les clients dont le client_id correspond à ct_0 et ct_1.

In [162]:
# suppression des valeurs dupliquées
df_transactions = df_transactions.drop_duplicates()
df_transactions[df_duplicated_transactions]

<ipython-input-162-e5e6209c4b04>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_transactions[df_duplicated_transactions]


,id_prod,date,session_id,client_id


Les valeurs dupliquées sont supprimées. on va verifier s'il reste encore des transactions avec identifiant s_0  a  une date contenant "test"

In [163]:
df_transactions_s_0 = df_transactions[df_transactions['date'].str.contains('test')]
df_transactions_s_0

,id_prod,date,session_id,client_id
3019,T_0,test_2021-03-01 02:30:02.237419,s_0,ct_0
5138,T_0,test_2021-03-01 02:30:02.237425,s_0,ct_0
9668,T_0,test_2021-03-01 02:30:02.237437,s_0,ct_1
10728,T_0,test_2021-03-01 02:30:02.237436,s_0,ct_0
15292,T_0,test_2021-03-01 02:30:02.237430,s_0,ct_0
...,...,...,...,...
577324,T_0,test_2021-03-01 02:30:02.237424,s_0,ct_0
593065,T_0,test_2021-03-01 02:30:02.237422,s_0,ct_1
607892,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_0
626049,T_0,test_2021-03-01 02:30:02.237422,s_0,ct_0


In [164]:
# suppression des sessions s_0
df_transactions = df_transactions[~df_transactions['date'].str.contains('test')]
df_transactions

,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232
...,...,...,...,...
679527,0_1551,2022-01-15 13:05:06.246925,s_150195,c_8489
679528,1_639,2022-03-19 16:03:23.429229,s_181434,c_4370
679529,0_1425,2022-12-20 04:33:37.584749,s_314704,c_304
679530,0_1994,2021-07-16 20:36:35.350579,s_63204,c_2227


In [165]:
df_transactions.loc[df_transactions['session_id'] == 's_0']

,id_prod,date,session_id,client_id


Nous avons supprimer toutes les données issues des tests, toutes les sessions dont l'identifiant session vaut 's_0'.

In [166]:
# Analyse des types de chaque variable
df_transactions.dtypes

id_prod       object
date          object
session_id    object
client_id     object
dtype: object

Nous remarquons que la variable "date" est déclaré comme  objet, nous allons la transformé en type 'datetime'.

In [167]:
# changement de type de la variable "date"
df_transactions['date'] = pd.to_datetime(df_transactions['date'])
print(df_transactions.dtypes)

id_prod               object
date          datetime64[ns]
session_id            object
client_id             object
dtype: object


## Nettoyage du fichier des produits

In [168]:
# les 5 premiers lignes du fichier des produits
df_products.head()

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0


In [169]:
# les données manquantes du fichier produits
df_products.isna().sum()

id_prod    0
price      0
categ      0
dtype: int64

Le fichier ne présente pas de données manquantes.

In [170]:
# verification des doublons de la clé primaire id_prod
df_products.duplicated('id_prod').sum()

0

Le fichier ne contient pas d'identifiant de produit dupliqué

In [171]:
# Analyse des types de chaque variable
df_products.dtypes

id_prod     object
price      float64
categ        int64
dtype: object

In [172]:
print('Modalités de \'categ\' :', list(df_products['categ'].unique()))

Modalités de 'categ' : [0, 1, 2]


La variable 'categ' compte 3 modalités : 0, 1 et 2.

In [173]:
# Reorganisation de la dataframe par categorie
df_products = df_products.sort_values('categ')
df_products

,id_prod,price,categ
0,0_1421,19.99,0
1985,0_724,21.78,0
1986,0_1467,4.99,0
1988,0_1076,25.11,0
1989,0_2211,9.99,0
...,...,...,...
719,2_223,175.99,2
1272,2_7,69.99,2
1729,2_84,182.30,2
1284,2_211,203.99,2


La table produits détaille l'identifiant, le prix et la catégorie de 3287 livres.

Lors du nettoyage du fichier des transactions, nous avions supprimés toutes les sessions de test avec comme identifiant produit T_0. Nous allons faire de meme sur notre dataframe "products" pour ne pas fausser notre analyse.

In [174]:
df_products_T_0 = df_products.loc[df_products['id_prod'] == 'T_0']
df_products_T_0

,id_prod,price,categ
731,T_0,-1.0,0


nous avons 731 produits avec l'identifiant T_0 avec un prix négatif qui est une valeur aberrante. Nous allons supprimer ces données de la dataframe.

In [175]:
df_products.drop(731, 0, inplace=True)


In [176]:
df_products_T_0 = df_products.loc[df_products['id_prod'] == 'T_0']
df_products_T_0

,id_prod,price,categ


nous remarquons que les données ont été supprimées du fichier des produits

In [177]:
# Reorganisation de la dataframe par categorie
df_products

,id_prod,price,categ
0,0_1421,19.99,0
1985,0_724,21.78,0
1986,0_1467,4.99,0
1988,0_1076,25.11,0
1989,0_2211,9.99,0
...,...,...,...
719,2_223,175.99,2
1272,2_7,69.99,2
1729,2_84,182.30,2
1284,2_211,203.99,2


La table produits présente maintenant l'identifiant, le prix et la catégorie de 3286 livres

## Nettoyage du fichier des clients

In [178]:
# Visualisation des lignes du fichier des clients
df_customers

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943
...,...,...,...
8618,c_7920,m,1956
8619,c_7403,f,1970
8620,c_5119,m,1974
8621,c_5643,f,1968


le fichier recense l'identifiant, le sexe et l'année de naissance de 8623 individus

In [179]:
# On affiche ensuite les données manquantes du fichier
df_customers.isna().sum()

client_id    0
sex          0
birth        0
dtype: int64

Le fichier ne présente pas de données manquantes.

In [180]:
# verification des doublons de la clé primaires client_id
df_customers.duplicated('client_id').sum()

0

les 8623 clients sont tous distincts.

In [181]:
# Analyse des types de chaque variable
df_customers.dtypes

client_id    object
sex          object
birth         int64
dtype: object

In [182]:
print('Modalités de \'sex\' :', list(df_customers['sex'].unique()))

Modalités de 'sex' : ['f', 'm']


La variable 'sex' compte 2 modalités : f et m (feminin et masculin).

Comme pour le fichier produits, nous allons supprimer du fichier client tous identifiants (ct_0 et ct_1) de la session de test.

In [183]:
df_customers_ct_0 = df_customers.loc[df_customers['client_id'] == 'ct_0']
df_customers_ct_1 = df_customers.loc[df_customers['client_id'] == 'ct_1']
print(df_customers_ct_0)
print(df_customers_ct_1)

     client_id sex  birth
2735      ct_0   f   2001
     client_id sex  birth
8494      ct_1   m   2001


Nous avons 2735 iterations pour l'identifiant ct_0 et 8484 pour l'identifiant ct_1

In [184]:
df_customers.drop([2735, 8494], 0, inplace=True)

In [185]:
# Visualisation de la dataset Clients
df_customers

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943
...,...,...,...
8618,c_7920,m,1956
8619,c_7403,f,1970
8620,c_5119,m,1974
8621,c_5643,f,1968


Aprés nettoyage, le fichier recense 8621 individus

# Mission_2 : Jointure des differentes dataframes

In [186]:
# merger les 2 tableaux 
df_transactions_products = pd.merge(df_transactions,df_products,how="outer", on='id_prod',indicator=True)
df_transactions_products

,id_prod,date,session_id,client_id,price,categ,_merge
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,4.18,0.0,both
1,0_1518,2021-09-26 12:37:29.780414,s_95811,c_6197,4.18,0.0,both
2,0_1518,2021-05-06 17:14:43.117440,s_30782,c_682,4.18,0.0,both
3,0_1518,2022-03-16 18:57:10.420103,s_180057,c_5932,4.18,0.0,both
4,0_1518,2022-11-12 18:58:10.574853,s_296584,c_7217,4.18,0.0,both
...,...,...,...,...,...,...,...
679348,1_394,NaT,NaN,NaN,39.73,1.0,right_only
679349,1_0,NaT,NaN,NaN,31.82,1.0,right_only
679350,2_72,NaT,NaN,NaN,141.32,2.0,right_only
679351,2_87,NaT,NaN,NaN,220.99,2.0,right_only


In [187]:
# Analyse des tableaux merger
df_merge_right_only = df_transactions_products.loc[df_transactions_products['_merge'] == 'right_only']
df_merge_right_only

,id_prod,date,session_id,client_id,price,categ,_merge
679332,0_310,NaT,NaN,NaN,1.94,0.0,right_only
679333,0_322,NaT,NaN,NaN,2.99,0.0,right_only
679334,0_1620,NaT,NaN,NaN,0.80,0.0,right_only
679335,0_1025,NaT,NaN,NaN,24.99,0.0,right_only
679336,0_510,NaT,NaN,NaN,23.66,0.0,right_only
679337,0_2308,NaT,NaN,NaN,20.28,0.0,right_only
679338,0_1624,NaT,NaN,NaN,24.50,0.0,right_only
679339,0_299,NaT,NaN,NaN,22.99,0.0,right_only
679340,0_1014,NaT,NaN,NaN,1.15,0.0,right_only
679341,0_1062,NaT,NaN,NaN,20.08,0.0,right_only


Ce sont des produits invendus, donc liés à aucune transaction. On peut les supprimer de la dataframe.

In [188]:
df_transactions_products = df_transactions_products.loc[df_transactions_products['_merge'] != 'right_only']
df_transactions_products

,id_prod,date,session_id,client_id,price,categ,_merge
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,4.18,0.0,both
1,0_1518,2021-09-26 12:37:29.780414,s_95811,c_6197,4.18,0.0,both
2,0_1518,2021-05-06 17:14:43.117440,s_30782,c_682,4.18,0.0,both
3,0_1518,2022-03-16 18:57:10.420103,s_180057,c_5932,4.18,0.0,both
4,0_1518,2022-11-12 18:58:10.574853,s_296584,c_7217,4.18,0.0,both
...,...,...,...,...,...,...,...
679327,0_1279,2021-06-23 06:50:48.888907,s_52563,c_1025,20.99,0.0,both
679328,0_1683,2021-12-24 04:37:46.407713,s_139091,c_5508,2.99,0.0,both
679329,0_1379,2021-10-16 09:19:09.546713,s_105547,c_5425,2.99,0.0,both
679330,0_886,2021-03-17 03:34:14.461358,s_7410,c_7219,21.82,0.0,both


In [189]:
# Analyse des tableaux merger
df_merge_left_only = df_transactions_products.loc[df_transactions_products['_merge'] == 'left_only']
df_merge_left_only

,id_prod,date,session_id,client_id,price,categ,_merge
542560,0_2245,2022-09-23 07:22:38.636773,s_272266,c_4746,NaN,NaN,left_only
542561,0_2245,2022-07-23 09:24:14.133889,s_242482,c_6713,NaN,NaN,left_only
542562,0_2245,2022-12-03 03:26:35.696673,s_306338,c_5108,NaN,NaN,left_only
542563,0_2245,2021-08-16 11:33:25.481411,s_76493,c_1391,NaN,NaN,left_only
542564,0_2245,2022-07-16 05:53:01.627491,s_239078,c_7954,NaN,NaN,left_only
...,...,...,...,...,...,...,...
542776,0_2245,2021-08-25 09:06:03.504061,s_80395,c_131,NaN,NaN,left_only
542777,0_2245,2022-03-06 19:59:19.462288,s_175311,c_4167,NaN,NaN,left_only
542778,0_2245,2022-05-16 11:35:20.319501,s_209381,c_4453,NaN,NaN,left_only
542779,0_2245,2022-02-11 09:05:43.952857,s_163405,c_1098,NaN,NaN,left_only


In [190]:
print(df_merge_left_only['id_prod'].unique())
print(df_merge_left_only['categ'].unique())
print(df_merge_left_only['price'].unique())

['0_2245']
[nan]
[nan]


On constate que le produit 0_2245 n'a pas de prix et n'est lié à aucune categorie. il faut choisir une méthode pour traiter ces valeurs manquantes.

Pour gérer les valeurs manquantes, 3 méthodes sont envisageables :

- Supprimer les lignes concernées
- Déterminer une valeur fixe
- Modéliser cette valeur

In [191]:
product_0_2245 = df_merge_left_only # Échantillon du produit 0_2245

# on peut calculer le pourcentage des ventes du produit 0_2245
pourcentage_vente_2245 = round((len(product_0_2245) / len(df_transactions_products) * 100), 2)
print('le produit 0_2245 represente', pourcentage_vente_2245, '% du dataset' )

le produit 0_2245 represente 0.03 % du dataset


Nous avons un faible taux de représentativité du produit 0_2245 dans le dataset. Nous pouvons le supprimer sans fausser l'analyse.


Nous pouvons également déterminer une valeur fixe pour les 2 variables. 


En regardant les prefixes des autres produits (0_, 1_ et 2_), ils correspondent invariablement à la catégorie du produit concerné. De ce fait nous choisissons 0 comme catégorie du produit 0_2245.

In [192]:
# Remplacement des valeurs manquantes de la variable 'categ' par 0
df_transactions_products['categ'].replace(np.nan, 0, inplace=True)
product_0_2245 = df_transactions_products[df_transactions_products['id_prod']=='0_2245']
product_0_2245

C:\Users\p092279\Anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,id_prod,date,session_id,client_id,price,categ,_merge
542560,0_2245,2022-09-23 07:22:38.636773,s_272266,c_4746,NaN,0.0,left_only
542561,0_2245,2022-07-23 09:24:14.133889,s_242482,c_6713,NaN,0.0,left_only
542562,0_2245,2022-12-03 03:26:35.696673,s_306338,c_5108,NaN,0.0,left_only
542563,0_2245,2021-08-16 11:33:25.481411,s_76493,c_1391,NaN,0.0,left_only
542564,0_2245,2022-07-16 05:53:01.627491,s_239078,c_7954,NaN,0.0,left_only
...,...,...,...,...,...,...,...
542776,0_2245,2021-08-25 09:06:03.504061,s_80395,c_131,NaN,0.0,left_only
542777,0_2245,2022-03-06 19:59:19.462288,s_175311,c_4167,NaN,0.0,left_only
542778,0_2245,2022-05-16 11:35:20.319501,s_209381,c_4453,NaN,0.0,left_only
542779,0_2245,2022-02-11 09:05:43.952857,s_163405,c_1098,NaN,0.0,left_only


Concernant le prix, on peut se baser sur les indicateurs de la tendance centrale de prix dans la catégorie 0.

In [193]:
df_transactions_products[df_transactions_products['categ']==0]['price'].agg([np.mean, np.std, np.median, np.min, np.max])

mean      10.638188
std        4.933527
median     9.990000
amin       0.620000
amax      40.990000
Name: price, dtype: float64

Nous constatons qu'il n'ya pas une grande différence entre la moyenne et la mediane, mais étant donné que la distribution du prix pour la categorie 0 est uniforme, nous retiendrons le prix moyen (10.63) comme valeur de remplacement.

In [194]:
df_transactions_products['price'].replace(np.nan, 10.63, inplace=True)
product_0_2245 = df_transactions_products[df_transactions_products['id_prod']=='0_2245']
product_0_2245.head()

C:\Users\p092279\Anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,id_prod,date,session_id,client_id,price,categ,_merge
542560,0_2245,2022-09-23 07:22:38.636773,s_272266,c_4746,10.63,0.0,left_only
542561,0_2245,2022-07-23 09:24:14.133889,s_242482,c_6713,10.63,0.0,left_only
542562,0_2245,2022-12-03 03:26:35.696673,s_306338,c_5108,10.63,0.0,left_only
542563,0_2245,2021-08-16 11:33:25.481411,s_76493,c_1391,10.63,0.0,left_only
542564,0_2245,2022-07-16 05:53:01.627491,s_239078,c_7954,10.63,0.0,left_only


In [195]:
df_transactions_products = df_transactions_products.drop(columns=['_merge'])
df_transactions_products

,id_prod,date,session_id,client_id,price,categ
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,4.18,0.0
1,0_1518,2021-09-26 12:37:29.780414,s_95811,c_6197,4.18,0.0
2,0_1518,2021-05-06 17:14:43.117440,s_30782,c_682,4.18,0.0
3,0_1518,2022-03-16 18:57:10.420103,s_180057,c_5932,4.18,0.0
4,0_1518,2022-11-12 18:58:10.574853,s_296584,c_7217,4.18,0.0
...,...,...,...,...,...,...
679327,0_1279,2021-06-23 06:50:48.888907,s_52563,c_1025,20.99,0.0
679328,0_1683,2021-12-24 04:37:46.407713,s_139091,c_5508,2.99,0.0
679329,0_1379,2021-10-16 09:19:09.546713,s_105547,c_5425,2.99,0.0
679330,0_886,2021-03-17 03:34:14.461358,s_7410,c_7219,21.82,0.0


In [196]:
# merger les 2 tableaux 
df_final = pd.merge(df_transactions_products,df_customers,how="outer", on='client_id', indicator=True)
df_final

,id_prod,date,session_id,client_id,price,categ,sex,birth,_merge
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,4.18,0.0,f,1986,both
1,0_1518,2021-07-20 13:21:29.043970,s_64849,c_103,4.18,0.0,f,1986,both
2,0_1518,2022-08-20 13:21:29.043970,s_255965,c_103,4.18,0.0,f,1986,both
3,0_1418,2022-06-18 01:49:37.823274,s_225411,c_103,8.57,0.0,f,1986,both
4,0_1418,2021-08-18 01:49:37.823274,s_77214,c_103,8.57,0.0,f,1986,both
...,...,...,...,...,...,...,...,...,...
679348,NaN,NaT,NaN,c_862,NaN,NaN,f,1956,right_only
679349,NaN,NaT,NaN,c_7584,NaN,NaN,f,1960,right_only
679350,NaN,NaT,NaN,c_90,NaN,NaN,m,2001,right_only
679351,NaN,NaT,NaN,c_587,NaN,NaN,m,1993,right_only


In [197]:
# Analyse des tableaux merger
df_mergefinal_left_only = df_final.loc[df_final['_merge'] == 'left_only']
df_mergefinal_left_only

,id_prod,date,session_id,client_id,price,categ,sex,birth,_merge


In [198]:
# Analyse des tableaux merger
df_mergefinal_right_only = df_final.loc[df_final['_merge'] == 'right_only']
df_mergefinal_right_only

,id_prod,date,session_id,client_id,price,categ,sex,birth,_merge
679332,NaN,NaT,NaN,c_8253,NaN,NaN,f,2001,right_only
679333,NaN,NaT,NaN,c_3789,NaN,NaN,f,1997,right_only
679334,NaN,NaT,NaN,c_4406,NaN,NaN,f,1998,right_only
679335,NaN,NaT,NaN,c_2706,NaN,NaN,f,1967,right_only
679336,NaN,NaT,NaN,c_3443,NaN,NaN,m,1959,right_only
679337,NaN,NaT,NaN,c_4447,NaN,NaN,m,1956,right_only
679338,NaN,NaT,NaN,c_3017,NaN,NaN,f,1992,right_only
679339,NaN,NaT,NaN,c_4086,NaN,NaN,f,1992,right_only
679340,NaN,NaT,NaN,c_6930,NaN,NaN,m,2004,right_only
679341,NaN,NaT,NaN,c_4358,NaN,NaN,m,1999,right_only


Ces données correspondent à des clients qui n'ont pas effectués d'achats (pas de session_Id), ils sont inactifs. On peut les supprimer du dataset

In [199]:
df_final = df_final.loc[df_final['_merge'] != 'right_only']
df_final = df_final.drop(columns=['_merge'])
df_final

,id_prod,date,session_id,client_id,price,categ,sex,birth
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,4.18,0.0,f,1986
1,0_1518,2021-07-20 13:21:29.043970,s_64849,c_103,4.18,0.0,f,1986
2,0_1518,2022-08-20 13:21:29.043970,s_255965,c_103,4.18,0.0,f,1986
3,0_1418,2022-06-18 01:49:37.823274,s_225411,c_103,8.57,0.0,f,1986
4,0_1418,2021-08-18 01:49:37.823274,s_77214,c_103,8.57,0.0,f,1986
...,...,...,...,...,...,...,...,...
679327,2_147,2021-07-23 07:01:38.963669,s_65994,c_4391,181.99,2.0,f,2001
679328,0_142,2022-03-25 18:07:25.880052,s_184472,c_1232,19.85,0.0,f,1960
679329,0_142,2021-09-25 18:07:25.880052,s_95415,c_1232,19.85,0.0,f,1960
679330,2_205,2021-03-11 00:11:32.016264,s_4605,c_7534,100.99,2.0,m,1992


In [200]:
df_final.to_csv('C:/Users/p092279/Desktop/Documents/Data Analyst/P6/DAN-P6-donnees/df_final.csv', index=False)